# Data Analysis for 5001 Project

In [2]:
import pandas as pd
import numpy as np
import csv
import re
from ast import literal_eval
import warnings
warnings.filterwarnings("ignore")

## Part1: MovieLens dataset
* 1.处理ratings.dat生成ratings表
    * 读取数据集加入每列的column名
    * 把timestamp转化为时间格式
    * 保存为df_ratings
* 2.处理movies.dat生成movies表
    * 读取数据集加入每列的column名
    * 对电影名进行格式预处理
    * 从电影名中取出年份信息
    * 处理带括号的电影名
    * 保存为df_ML_movies
* 3.处理u.users.dat生成users表
    * 读取数据集加入每列的column名
    * 保存为df_users

### Table 1: ratings

In [242]:
ratings_path = "data/movielens/ratings.dat"
column_ratings = ["user_id", "movie_id_ml", "rating", "rating_timestamp"]
df_ratings = pd.read_csv(ratings_path, header=None, encoding="ISO-8859-1",names=column_ratings, delimiter="::", quoting=csv.QUOTE_NONE)

df_ratings["rating_timestamp"] = pd.to_datetime(df_ratings["rating_timestamp"], unit="s")

print(df_ratings.shape)
print(df_ratings.dtypes)
df_ratings.head()

(1000209, 4)
user_id                      int64
movie_id_ml                  int64
rating                       int64
rating_timestamp    datetime64[ns]
dtype: object


,user_id,movie_id_ml,rating,rating_timestamp
0,1,1193,5,2000-12-31 22:12:40
1,1,661,3,2000-12-31 22:35:09
2,1,914,3,2000-12-31 22:32:48
3,1,3408,4,2000-12-31 22:04:35
4,1,2355,5,2001-01-06 23:38:11


### Table2: movies

In [243]:
# 预处理电影名
def clean_movie_title(movie_title):
    if movie_title.split(" ")[-1].startswith("("):
        # remove year from the title, e.g. Toy Story (1995) --> Toy Story
        movie_title = (" ".join(movie_title.split(" ")[:-1])).strip()

    if movie_title.title().split(',')[-1].strip() in ['The', 'A']:
        # article + movie title, e.g. Saint, The --> The Saint
        movie_title = (movie_title.title().split(',')[-1].strip() + " " + " ".join(movie_title.title().split(',')[:-1])).strip()

    # otherwise, it was converting The Devil's Advocate to The Devil'S Advocate
    movie_title = movie_title.lower()
    return movie_title

In [244]:
# movies
movies_path = "data/movielens/movies.dat"
column_item = ["movie_id_ml", "title", "genre"]
df_ML_movies = pd.read_csv(movies_path, header=None, encoding="ISO-8859-1",names=column_item, delimiter="::", quoting=csv.QUOTE_NONE)

In [245]:
df_ML_movies

,movie_id_ml,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [246]:
df_ML_movies["release"] = df_ML_movies["title"].apply(lambda row : row[-5:-1])
# df_ML_movies["title"] = df_ML_movies["title"].apply(lambda row : row[0:-7])

In [247]:
df_ML_movies["title"] = df_ML_movies["title"].apply(lambda row : clean_movie_title(row))
# drop rows where movie starts with brackets, those are some strange names...
df_ML_movies = df_ML_movies[~df_ML_movies.title.str.startswith("(")]

# handle seven (se7en) movies, creating new rows containing the content of brackets
_df = df_ML_movies[df_ML_movies.title.str.contains("(", regex=False)]
_df.title = _df.title.apply(lambda x: re.search(r'\((.*?)\)', x).group(1).strip() if re.search(r'\((.*?)\)', x) else x.strip())
df_ML_movies = df_ML_movies.append(_df)

print(df_ML_movies.shape)
print(df_ML_movies.dtypes)
df_ML_movies.head()

(4151, 4)
movie_id_ml     int64
title          object
genre          object
release        object
dtype: object


,movie_id_ml,title,genre,release
0,1,toy story,Animation|Children's|Comedy,1995
1,2,jumanji,Adventure|Children's|Fantasy,1995
2,3,grumpier old men,Comedy|Romance,1995
3,4,waiting to exhale,Comedy|Drama,1995
4,5,father of the bride part ii,Comedy,1995


In [248]:
def split_col(data, columns):
    for c in columns:
        new_col = data.pop(c)
        max_len = max(list(map(lambda x:len(x) if isinstance(x, list) else 1, new_col.values))) # 最大长度
        new_col = new_col.apply(lambda x: x+[None]*(max_len - len(x)) if isinstance(x, list) else [x]+[None]*(max_len - 1)) # 补空值，None可换成np.nan
        new_col = np.array(new_col.tolist()).T # 转置
        for i, j in enumerate(new_col):
            data[c + str(i)] = j

In [249]:
split_col(df_ML_movies, columns = ["genre"])

In [250]:
df_ML_movies

,movie_id_ml,title,release,genre0
0,1,toy story,1995,Animation|Children's|Comedy
1,2,jumanji,1995,Adventure|Children's|Fantasy
2,3,grumpier old men,1995,Comedy|Romance
3,4,waiting to exhale,1995,Comedy|Drama
4,5,father of the bride part ii,1995,Comedy
...,...,...,...,...
3794,3864,gojira ni-sen mireniamu,1999,Action|Adventure|Sci-Fi
3797,3867,a.k.a. it's the rage,1999,Drama
3822,3892,anatomie,2000,Horror
3832,3902,goya en bodeos,1999,Drama


### Table3: users

In [4]:
users_path = "data/movielens/users.dat"
column_user = ["user_id", "user_gender", "user_age", "user_occupation", "user_zipcode"]
df_users = pd.read_csv(users_path, header=None, encoding="ISO-8859-1",names=column_user, delimiter="::", quoting=csv.QUOTE_NONE)

df_users.head()

,user_id,user_gender,user_age,user_occupation,user_zipcode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


## Part2: IMDb dataset
* 1.处理title.basics
    * 读取数据集
    * 添加title为primaryTitle, release为startYear
    * 对电影名进行格式预处理（与movielens中相同）
* 2.处理movies.dat生成movies表
    * 读取数据集加入每列的column名
    * 对电影名进行格式预处理
    * 从电影名中取出年份信息
    * 处理带括号的电影名
    * 保存为df_ML_movies
* 3.处理u.users.dat生成users表
    * 读取数据集加入每列的column名
    * 保存为df_users

### Table4: title.basics

In [252]:
basics_path = "data/imdb/title.basics.tsv.gz"
df_basics = pd.read_csv(basics_path,encoding="ISO-8859-1", sep="\t")

In [253]:
df_basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...
8827322,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2010,\N,\N,"Action,Drama,Family"
8827323,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,\N,\N,"Action,Drama,Family"
8827324,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,\N,"Action,Drama,Family"
8827325,tt9916856,short,The Wind,The Wind,0,2015,\N,27,Short


In [254]:
df_basics["release"] = df_basics["startYear"]
df_basics["title"] = df_basics["primaryTitle"]
df_basics["title"] = df_basics["title"].astype(str)
df_basics["title"] = df_basics["title"].apply(lambda x: x.lower())

In [255]:
df_basics_movie = df_basics[df_basics["titleType"] == "movie"]
df_basics_movie

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,release,title
498,tt0000502,movie,Bohemios,Bohemios,0,1905,\N,100,\N,1905,bohemios
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Action,Adventure,Biography",1906,the story of the kelly gang
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,\N,90,Drama,1907,the prodigal son
610,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,0,1907,\N,\N,Drama,1907,robbery under arms
625,tt0000630,movie,Hamlet,Amleto,0,1908,\N,\N,Drama,1908,hamlet
...,...,...,...,...,...,...,...,...,...,...,...
8827217,tt9916622,movie,Rodolpho TeÃ³philo - O Legado de um Pioneiro,Rodolpho TeÃ³philo - O Legado de um Pioneiro,0,2015,\N,57,Documentary,2015,rodolpho teã³philo - o legado de um pioneiro
8827244,tt9916680,movie,De la ilusiÃ³n al desconcierto: cine colombian...,De la ilusiÃ³n al desconcierto: cine colombian...,0,2007,\N,100,Documentary,2007,de la ilusiã³n al desconcierto: cine colombian...
8827256,tt9916706,movie,Dankyavar Danka,Dankyavar Danka,0,2013,\N,\N,Comedy,2013,dankyavar danka
8827267,tt9916730,movie,6 Gunn,6 Gunn,0,2017,\N,116,\N,2017,6 gunn


In [256]:
# drop rows where movie starts with brackets, those are some strange names...
df_basics_movie = df_basics_movie[~df_basics.title.str.startswith("(")]

# handle seven (se7en) movies, creating new rows containing the content of brackets
_df = df_basics_movie[df_basics_movie.title.str.contains("(", regex=False)]
_df.title = _df.title.apply(lambda x: re.search(r'\((.*?)\)', x).group(1).strip() if re.search(r'\((.*?)\)', x) else x.strip())
df_basics_movie = df_basics_movie.append(_df)

print(df_basics_movie.shape)
print(df_basics_movie.dtypes)
df_basics_movie.head()

(609454, 11)
tconst            object
titleType         object
primaryTitle      object
originalTitle     object
isAdult           object
startYear         object
endYear           object
runtimeMinutes    object
genres            object
release           object
title             object
dtype: object


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,release,title
498,tt0000502,movie,Bohemios,Bohemios,0,1905,\N,100,\N,1905,bohemios
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Action,Adventure,Biography",1906,the story of the kelly gang
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,\N,90,Drama,1907,the prodigal son
610,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,0,1907,\N,\N,Drama,1907,robbery under arms
625,tt0000630,movie,Hamlet,Amleto,0,1908,\N,\N,Drama,1908,hamlet


### Table5: title.principals

In [257]:
principals_path = "data/imdb/title.principals.tsv.gz"
df_principals = pd.read_csv(principals_path, sep="\t")

In [258]:
df_principals.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N


In [259]:
df_principals["category"].value_counts()

actor                  11060267
self                    8679029
actress                 8497760
writer                  6846908
director                5727872
producer                3200745
cinematographer         1719987
composer                1715085
editor                  1646282
production_designer      340973
archive_footage          317221
archive_sound              3148
Name: category, dtype: int64

In [ ]:
df_principals_group = df_principals.groupby("tconst")["nconst"].apply(lambda x :x.str.cat(sep = ",")).reset_index()

In [ ]:
df_principals_group

### Table6: title.ratings

In [41]:
imdb_ratings_path = "data/imdb/title.ratings.tsv.gz"
df_imdb_ratings = pd.read_csv(imdb_ratings_path, sep="\t")

In [42]:
df_imdb_ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1869
1,tt0000002,5.9,247
2,tt0000003,6.5,1644
3,tt0000004,5.8,159
4,tt0000005,6.2,2470
...,...,...,...
1231203,tt9916690,6.6,5
1231204,tt9916720,5.7,202
1231205,tt9916730,8.7,6
1231206,tt9916766,6.7,19


## Part3: Merge IMDb and MovieLens

### Merge All Movies Info

In [210]:
df_movies = pd.merge(df_ML_movies, df_basics_movie, on=["title", "release"])

In [213]:
df_movies = df_movies.rename(columns = {"genre": "movielens_genre", "genres": "imdb_genre"})

In [216]:
df_movies_prin = pd.merge(df_movies, df_principals_group, on=["tconst"])

In [218]:
df_movies_all = pd.merge(df_movies_prin, df_imdb_ratings, on=["tconst"])

In [220]:
df_movies_all = df_movies_all.rename(columns = {"nconst": "principals"})

In [221]:
df_movies_all

,movie_id_ml,title,movielens_genre,release,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,imdb_genre,principals,averageRating,numVotes
0,1,toy story,Animation|Children's|Comedy,1995,tt0114709,movie,Toy Story,Toy Story,0,1995,\N,81,"Adventure,Animation,Comedy","nm0169505,nm0000158,nm0000741,nm0725543,nm0001...",8.3,957798
1,2,jumanji,Adventure|Children's|Fantasy,1995,tt0113497,movie,Jumanji,Jumanji,0,1995,\N,104,"Adventure,Comedy,Family","nm0472256,nm0000245,nm0000379,nm0001372,nm0404...",7.0,334898
2,3,grumpier old men,Comedy|Romance,1995,tt0113228,movie,Grumpier Old Men,Grumpier Old Men,0,1995,\N,101,"Comedy,Romance","nm0005714,nm0000527,nm0000493,nm0000268,nm0000...",6.6,26964
3,4,waiting to exhale,Comedy|Drama,1995,tt0114885,movie,Waiting to Exhale,Waiting to Exhale,0,1995,\N,124,"Comedy,Drama,Romance","nm0004892,nm0001365,nm0000291,nm0222643,nm0005...",5.9,10795
4,5,father of the bride part ii,Comedy,1995,tt0113041,movie,Father of the Bride Part II,Father of the Bride Part II,0,1995,\N,106,"Comedy,Family,Romance","nm0204567,nm0000188,nm0000473,nm0001737,nm0931...",6.0,37456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2636,1734,ma vie en rose,Comedy|Drama,1997,tt0119590,movie,Ma vie en rose,Ma vie en rose,0,1997,\N,88,Drama,"nm0134992,nm0238878,nm0488477,nm0959897,nm0898...",7.5,8756
2637,2934,el amor brujo,Musical,1986,tt0090636,movie,El amor brujo,El amor brujo,0,1986,\N,100,"Drama,Music","nm0215811,nm0300565,nm0398417,nm0812556,nm0422...",7.0,893
2638,3192,malli,Drama,1998,tt0293385,movie,Malli,Malli,0,1998,\N,90,Drama,"nm1451303,nm5878059,nm0007144,nm0221219,nm2201...",7.9,55
2639,3407,les convoyeurs attendent,Comedy|Drama,1999,tt0201538,movie,Les convoyeurs attendent,Les convoyeurs attendent,0,1999,\N,94,"Comedy,Drama","nm0762903,nm0688143,nm0800311,nm0099521,nm0046...",6.8,1295


In [222]:
df_movies_all.to_csv("data/final_data/movies.csv", index = None, encoding="utf8")

### Generate ratings dataset

In [232]:
df_movies_id = pd.DataFrame(df_movies_all["movie_id_ml"])

In [234]:
df_ratings_new = pd.merge(df_ratings, df_movies_id, on=["movie_id_ml"])

In [235]:
df_ratings_new

,user_id,movie_id_ml,rating,rating_timestamp
0,1,1193,5,2000-12-31 22:12:40
1,2,1193,5,2000-12-31 21:33:33
2,12,1193,4,2000-12-30 23:49:39
3,15,1193,4,2000-12-30 18:01:19
4,17,1193,5,2000-12-30 06:41:11
...,...,...,...,...
796032,5556,2198,3,2000-05-27 16:38:35
796033,5949,2198,5,2000-05-20 18:13:21
796034,5675,2703,3,2000-12-05 15:11:56
796035,5851,3607,5,2000-05-08 03:30:08


In [236]:
df_ratings_new.to_csv("data/final_data/ratings.csv", index = None, encoding="utf8")

### Generate users dataset

In [5]:
df_users.to_csv("data/final_data/users.csv", index = None, encoding="utf8")

## Part4: Generate Profiling Report

In [7]:
import pandas_profiling

In [228]:
profile_movies = df_movies_all.profile_report(title='Movies Report')
profile_movies.to_file(output_file="report/movies_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [237]:
profile_ratings = df_ratings_new.profile_report(title='Ratings Report')
profile_ratings.to_file(output_file="report/ratings_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
profile_users = df_users.profile_report(title='Users Report')
profile_users.to_file(output_file="report/users_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]